In [ ]:
# %% [cell]
# ProjectTwoDashboard.ipynb - Full code for dashboard
# Setup the Jupyter version of Dash and imports
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash import dash_table
from dash.dependencies import Input, Output, State
import dash_leaflet as dl
import plotly.express as px
import base64
import pandas as pd
import numpy as np
import os

JupyterDash.infer_jupyter_proxy_config()

# Import your CRUD module (as requested)
from animal_shelter import AnimalShelter

# %% [cell]
# ------------------------------
# Data / Model: connect to MongoDB
# ------------------------------
username = "aacuser"
password = "StrongPassword123"   # as requested
host = "localhost"
port = 27017
db_name = "aac"
col_name = "animals"

# instantiate your CRUD object
db = AnimalShelter(user=username, password=password, host=host, port=port, db=db_name, col=col_name)

# pull all records (empty query)
records = db.read({})

# convert to DataFrame and clean
df = pd.DataFrame.from_records(records)
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# If there are no rows, create an empty dataframe with expected columns to prevent crashes
if df.empty:
    # define minimal expected columns (adjust as needed)
    df = pd.DataFrame(columns=[
        'animal_type','name','breed','age_upon_outcome','outcome_type',
        'datetime','date_of_birth','color','sex_upon_outcome','outcome_subtype',
        'monthyear','rec_num','animal_id','location_lat','location_long'
    ])

# %% [cell]
# ------------------------------
# App & Layout
# ------------------------------
app = JupyterDash(__name__)

# Logo: change image_filename if your logo filename differs
image_filename = "Grazioso_Salvare_Logo.png"  # put your logo file in same folder
encoded_image = None
if os.path.exists(image_filename):
    encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

app.layout = html.Div([
    # Header with logo and unique identifier (your name)
    html.Div([
        html.Div(
            [
                html.Img(src=f'data:image/png;base64,{encoded_image}', style={'height': '80px'}) 
                if encoded_image else html.Div(), 
            ],
            style={'display': 'inline-block', 'verticalAlign': 'middle', 'marginRight': '20px'}
        ),
        html.Div(
            [
                html.H1("Grazioso Salvare Dashboard", style={'margin': '0'}),
                html.H4("Developer: Donovan Taylor", style={'margin': '0', 'color': '#666'})
            ],
            style={'display': 'inline-block', 'verticalAlign': 'middle'}
        )
    ], style={'textAlign': 'left', 'padding': '10px'}),

    html.Hr(),

    # Filter controls
    html.Div([
        html.Label("Rescue Type Filter:"),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset (All Data)', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'inline-block', 'marginRight': '15px'}
        )
    ], style={'padding': '10px'}),

    html.Hr(),

    # Data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_action='native',
        page_current=0,
        page_size=10,
        filter_action='native',
        sort_action='native',
        sort_mode='multi',
        column_selectable='single',
        row_selectable='single',
        selected_rows=[0],  # default selects first row
        style_table={'overflowX': 'auto'},
    ),

    html.Br(),

    # Charts row: left chart and map on right
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6', style={'flex': '1', 'padding': '10px'}),
        html.Div(id='map-id', className='col s12 m6', style={'flex': '1', 'padding': '10px'})
    ])
], style={'margin': '10px'})

# %% [cell]
# ------------------------------
# Helper: rescue-type -> query mapping
# ------------------------------
def rescue_type_query(filter_type):
    # these breed lists are examples; adjust to match "Dashboard Specs" if required
    if filter_type == "Water Rescue":
        # example water rescue breeds
        breeds = ["Labrador Retriever Mix", "Newfoundland", "Chesapeake Bay Retriever", "Golden Retriever"]
    elif filter_type == "Mountain or Wilderness Rescue":
        breeds = ["German Shepherd", "Alaskan Malamute", "Siberian Husky", "Border Collie"]
    elif filter_type == "Disaster or Individual Tracking":
        breeds = ["Bloodhound", "German Shepherd", "Rottweiler", "Labrador Retriever Mix"]
    else:  # Reset or unknown
        return {}

    return {"animal_type": "Dog", "breed": {"$in": breeds}}

# %% [cell]
# ------------------------------
# Callback: update data table based on filter (controller)
# Output is datatable data list of dicts
# ------------------------------
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    # Build MongoDB query based on filter_type
    q = rescue_type_query(filter_type)
    try:
        filtered = db.read(q)
    except Exception as e:
        # if DB read fails, return current full df
        return df.to_dict('records')

    dff = pd.DataFrame.from_records(filtered)
    if '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True)

    # ensure same columns order as original df (fill missing cols)
    for col in df.columns:
        if col not in dff.columns:
            dff[col] = None
    dff = dff[df.columns]

    return dff.to_dict('records')

# %% [cell]
# ------------------------------
# Callback: update pie chart (graph-id) from datatable current view
# Input: derived_virtual_data (current table data after filtering / sorting / paging)
# ------------------------------
@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data')]
)
def update_graphs(viewData):
    if viewData is None:
        return [html.Div("No data available for chart.")]

    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return [html.Div("No data available for chart.")]

    # we will show a pie chart of breed counts (top 10)
    # guard for missing 'breed' column
    if 'breed' not in dff.columns:
        return [html.Div("Breed column not available for chart.")]

    # create summary
    pie_df = dff['breed'].fillna("Unknown").value_counts().reset_index()
    pie_df.columns = ['breed', 'count']
    pie_df = pie_df.head(10)

    fig = px.pie(pie_df, values='count', names='breed', title='Top Breeds (current view)')
    return [dcc.Graph(figure=fig)]

# %% [cell]
# ------------------------------
# Callback: style selected columns highlight
# ------------------------------
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{
        'if': {'column_id': i},
        'backgroundColor': '#D2F3FF'
    } for i in selected_columns]

# %% [cell]
# ------------------------------
# Callback: update map based on selected row in datatable
# Inputs: derived_virtual_data, derived_virtual_selected_rows
# ------------------------------
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, selected_rows):
    # if no data passed, return empty placeholder
    if viewData is None:
        return [html.Div("No data to map.")]

    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return [html.Div("No data to map.")]

    # Ensure lat/long columns exist — try common column names
    lat_col = None
    lon_col = None
    for candidate in ['location_lat', 'location_long', 'latitude', 'longitude', 'lat', 'long']:
        if candidate in dff.columns:
            if 'lat' in candidate:
                lat_col = candidate
            elif 'long' in candidate or 'lon' in candidate:
                lon_col = candidate

    # fallback to known column names used in dataset
    if lat_col is None and 'location_lat' in dff.columns:
        lat_col = 'location_lat'
    if lon_col is None and 'location_long' in dff.columns:
        lon_col = 'location_long'

    # ensure we have both
    if lat_col is None or lon_col is None:
        return [html.Div("Latitude/Longitude columns not found in data.")]

    # choose row index (single selection)
    if selected_rows is None or len(selected_rows) == 0:
        row_idx = 0
    else:
        row_idx = selected_rows[0]

    # guard row index bounds
    if row_idx < 0 or row_idx >= len(dff):
        row_idx = 0

    # attempt to coerce coordinates to float
    try:
        lat = float(dff.iloc[row_idx][lat_col])
        lon = float(dff.iloc[row_idx][lon_col])
    except Exception:
        # fallback default location (Austin)
        lat, lon = 30.75, -97.48

    # Tooltip + popup fields
    breed = dff.iloc[row_idx].get('breed', 'Unknown')
    name = dff.iloc[row_idx].get('name', 'Unknown')

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[lat, lon], zoom=12, children=[
                   dl.TileLayer(id="base-layer-id"),
                   dl.Marker(position=[lat, lon],
                             children=[
                                 dl.Tooltip(breed),
                                 dl.Popup([
                                     html.H4("Animal Name"),
                                     html.P(str(name))
                                 ])
                             ])
               ])
    ]

# %% [cell]
# ------------------------------
# Run app (inline)
# ------------------------------
# if 8050 is in use, change port or run without specifying port
app.run_server(mode='inline', port=8050)
